Randomized SVD. Draw $X \in \mathbb{R}^{m \times m}$ and $Y \in \mathbb{R}^{n \times }$ with $m \geq n$ uniformly at random
from the space of orthonormal matrices with $m = 1000$ and $n = 100000$. That is, X and Y
uniformly at random from $X^T X = I$ and $Y^TY = I$ e.g. in matlab $X=orth(randn(m,m))$
and $Y=orth(randn(n,m))$. Also set the diagonal matrix $D \in R^{m \times m}$ with diagonal entries

$$
D_{ii} = \begin{cases} r − i + 1 & \text{if } i \leq r\\
4 * 10^{−3} & \text{if } i > r
\end{cases}
$$

with $r = 10$. Set $A \in R^{m \times n}$ equal to $A = XDY^T$ . Use the randomized SVD of Drineas
et al. to calculate the r = 10 top eigenvectors of A. Let $A = U\Sigma V^T$ with $U \in R^{m \times m}$,
$V \in R^{n\times n}$ , and $\Sigma \in \mathbb{R}^{m \times n}$ be the singular value decomposition of A. 
Also let $U_r$ and $V_r$ be the singular vectors corresponding to the top $r$ singular values (if the singular values are
ordered in decreasing order this corresponds to the first $r$ columns of $U$ and $V$ ).


(i) How large does $c$ (number of drawn columns/rows) has to be so we can calculate the eigenvectors with relative errors of $\epsilon = 0.01, 0.05, 0.1$ That is,

$$\frac{||\hat{U}_r \hat{U}_r^T − U_rU_r^T||}{||U_rU_r^T||} \leq \epsilon$$

and 

$$\frac{||\hat{V}_r \hat{V}_r^T − V_rV_r^T||}{||V_rV_r^T||} \leq \epsilon$$

Report this number based on the average of 10 random draws from the columns of A.
That is for the matrix A draw $c$ columns 10 independent times. Compare the average
run time (over the 10 random draws) with that of using an SVD on A. Would you use
the randomized SVD algorithm?

Clarification: In the algorithm of Drineas et. al. to estimate $U_r$ you draw columns at
random and to estimate $V_r$ you draw rows at random.


Hint: Note that $||U_r U_r^T|| = ||V_r V_r^T|| = 1$. Also note that to calculate $||\hat{U}_r \hat{U}_r^T - \hat{U}_r \hat{U}_r^T||$
and $||\hat{V}_r \hat{V}_r^T - V_r V_r||$ you don’t need to build the matrices explicitly (in which case
you probably get an out of memory error). You can calculate the spectral norm of
$\hat{U}_r \hat{U}_r^T - U_r U_r^T$ via the power method by calculating matrix-vector products of the form
$(\hat{U}_r \hat{U}_r^T - U_r U_r^T) z$ which can be carried out efficiently. However, note that to calculate
the spectrum of the matrix $\hat{U}_r \hat{U}_r^T - U_r U_r^T$ you should probably apply the power method
using $(\hat{U}_r \hat{U}_r^T - U_r U_r^T)^2$ as the power method directly on $\hat{U}_r \hat{U}_r^T - U_r U_r^T$ will fail. Do
you know why?


In [1]:
%pylab inline
from scipy.linalg import svd
from numpy.linalg import norm
from scipy.sparse.linalg import svds, eigs
from sklearn.decomposition import TruncatedSVD
m = 1000
n = 100000
r = 10
epsilon = [0.01, 0.05, 0.1]

for eps in epsilon:
    for c in range(10, 100):
        
    
def power_method(Xr , eps=1e-6, n_steps=1000):
    z_k = np.random.rand(Xr.T.shape[1])
    z_k = z_k[:, np.newaxis]
    for _ in range(n_steps):
        # calculate the matrix-by-vector product Ab
        Xr_T_z_k1 = np.dot(Xr.T, z_k)
        Xr_Xr_T_z_k1 = np.dot(Xr, Xr_T_z_k1)

        # calculate the norm
        Xr_Xr_T_z_k1_norm = np.linalg.norm(Xr_Xr_T_z_k1)

        # re normalize the vector
        z_k = Xr_Xr_T_z_k1 / Xr_Xr_T_z_k1_norm
        z_diff = np.linalg.norm(z_k - Xr_Xr_T_z_k1)

    return z_k

def power_method_ur(Xr_hat, Xr , eps=1e-6, n_steps=1000):
    z_k = np.random.rand(Xr.T.shape[1])
    z_k = z_k[:, np.newaxis]
    for _ in range(n_steps):
        # calculate the matrix-by-vector product Ab
        z_k1 = np.dot(Xr.T, z_k)
        z_k1 = np.dot(Xr, z_k1)
        
        z_k1_hat = np.dot(Xr_hat.T, z_k)
        z_k1_hat = np.dot(Xr_hat, z_k1_hat)
        
        z_k1_diff = z_k1_hat - z_k1_hat
        # calculate the norm
        z_k1_norm = np.linalg.norm(z_k1_diff)

        # re normalize the vector
        z_k = z_k1_diff / z_k1_norm
        z_succ_diff = np.linalg.norm(z_k - z_k1_diff)

    return z_k

Populating the interactive namespace from numpy and matplotlib


/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
XF = np.random.random(size=(m, m))
X, _ = np.linalg.qr(XF)

YF = np.random.random(size=(n, m))
Y, _ = np.linalg.qr(YF)


In [3]:
x = [r-i+1 for i in range(0, r)] + [4*0.001 for i in range(r, m)]
x = np.array(x)


D = np.diag(x)

In [4]:
A = np.matmul(np.matmul(X,D), Y.T)

In [5]:
A.shape

(1000, 100000)

In [6]:
%time u, s, vt = svds(A, k=r)

CPU times: user 17.7 s, sys: 687 ms, total: 18.4 s
Wall time: 1.15 s


In [7]:
norm(A, 'fro')

22.47255748685494

In [8]:
col_probabilities = np.power(norm(A, 2, 0),2)/norm(A, 'fro')**2
                

In [9]:
col_probabilities.sum()

1.0000000000000002

In [10]:
norm(A, 'fro')

22.47255748685494

In [11]:
def power_iteration(A, eps=1e-3, n_steps=1000):
    z_k = np.random.rand(A.shape[1])

    for _ in range(n_steps):
        # calculate the matrix-by-vector product Ab
        z_k1 = np.dot(A, z_k)

        # calculate the norm
        z_k1_norm = np.linalg.norm(z_k1)

        # re normalize the vector
        z_k = z_k1 / z_k1_norm
        z_diff = np.linalg.norm(z_k - z_k1)
        if z_diff<=eps:
            print('converged')
            return z_k
    return z_k

In [12]:
# Ur
B = []
c = 10
col_probabilities = np.power(norm(A, 2, 0),2)/norm(A, 'fro')**2
col_choices = range(0, n)                
cols_to_sample = np.random.choice(col_choices, c, p=col_probabilities)
B = A[:, cols_to_sample]
Ur_hat, U_singular_values_hat, Ur_VrT_hat = svd(B)

In [13]:
B.shape

(1000, 10)

In [14]:
Ur_hat.shape

(1000, 1000)

In [15]:
U_singular_values_hat

array([0.13885367, 0.11394122, 0.09235962, 0.07935214, 0.05723284,
       0.04234487, 0.03922273, 0.01477933, 0.01010153, 0.00352601])

In [16]:
Ur_VrT_hat.shape

(10, 10)

In [17]:
# Vr
B = []
c = 10
row_probabilities = np.power(norm(A, 2, 1),2)/norm(A, 'fro')**2
row_choices = range(0, m)                
rows_to_sample = np.random.choice(row_choices, c, p=row_probabilities)


In [18]:
rows_to_sample

array([195, 312, 839, 888, 789, 400, 624, 797, 571,  55])

In [19]:
B = A[rows_to_sample, :]
B.shape

(10, 100000)

In [20]:
min(B.shape)

10

In [21]:
B = A[rows_to_sample, :]
Vr_hat, Vr_singular_values_hat, Vr_VrT_hat= svd(B)

In [22]:
B.shape

(10, 100000)

In [23]:
Vr_hat.shape

(10, 10)

In [24]:
Vr_VrT_hat.shape

(100000, 100000)

In [53]:
power_method(u)

array([[0.04046504],
       [0.02799995],
       [0.0366406 ],
       [0.03360569],
       [0.04275456],
       [0.03744584],
       [0.02955998],
       [0.01916776],
       [0.02205977],
       [0.03111676],
       [0.02944704],
       [0.02584871],
       [0.04212941],
       [0.0341794 ],
       [0.03125053],
       [0.02578014],
       [0.03833959],
       [0.03924591],
       [0.02704306],
       [0.03161332],
       [0.03978043],
       [0.03055999],
       [0.04387446],
       [0.02695185],
       [0.02427321],
       [0.03170022],
       [0.03201317],
       [0.0286262 ],
       [0.03706138],
       [0.03263229],
       [0.0317318 ],
       [0.02927835],
       [0.03203491],
       [0.02604192],
       [0.03064889],
       [0.02886487],
       [0.03770324],
       [0.03159686],
       [0.02958781],
       [0.03021843],
       [0.02400643],
       [0.03761866],
       [0.03351337],
       [0.02593782],
       [0.03612336],
       [0.01806509],
       [0.0297413 ],
       [0.032

In [32]:
Ur_hat[:, :10].shape

(1000, 10)

In [33]:
np.amax(np.linalg.svd(np.matmul(Ur_hat[:, :r], Ur_hat[:, :r].T)-np.matmul(u, u.T), compute_uv=0))

0.11210921832439423

In [54]:
ut,_, u_vt = np.linalg.svd(np.matmul(u, u.T))

In [64]:
ut,_, u_vt = np.linalg.svd(np.power(u,2))

In [65]:
np.round(ut[:,0],2)

array([-0.04, -0.04, -0.03, -0.02, -0.05, -0.02, -0.04, -0.03, -0.02,
       -0.02, -0.04, -0.04, -0.04, -0.03, -0.04, -0.03, -0.03, -0.04,
       -0.03, -0.03, -0.03, -0.02, -0.02, -0.03, -0.03, -0.03, -0.03,
       -0.03, -0.04, -0.03, -0.03, -0.02, -0.05, -0.03, -0.02, -0.03,
       -0.04, -0.02, -0.03, -0.03, -0.03, -0.03, -0.02, -0.05, -0.02,
       -0.02, -0.03, -0.04, -0.03, -0.05, -0.05, -0.04, -0.03, -0.03,
       -0.01, -0.03, -0.02, -0.03, -0.02, -0.01, -0.03, -0.04, -0.03,
       -0.03, -0.02, -0.02, -0.04, -0.02, -0.01, -0.03, -0.02, -0.04,
       -0.03, -0.04, -0.05, -0.03, -0.03, -0.03, -0.04, -0.04, -0.03,
       -0.03, -0.03, -0.03, -0.02, -0.01, -0.04, -0.03, -0.02, -0.01,
       -0.03, -0.03, -0.04, -0.03, -0.05, -0.03, -0.04, -0.02, -0.03,
       -0.04, -0.03, -0.04, -0.03, -0.04, -0.03, -0.04, -0.04, -0.02,
       -0.03, -0.03, -0.02, -0.03, -0.04, -0.02, -0.03, -0.04, -0.02,
       -0.03, -0.03, -0.03, -0.04, -0.03, -0.02, -0.04, -0.02, -0.04,
       -0.04, -0.03,